In [2]:
import numpy as np
import pandas as pd
import os

In [2]:
answer_df = pd.read_csv('./두통 답변.csv', index_col=0)
question_df = pd.read_csv('./두통 질문.csv', index_col=0)

In [3]:
answer_df

,disease_category,disease_name,intention,answer_intro_conclusion,answer
0,응급질환,두통,예방,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...
1,응급질환,두통,예방,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 마지막으로, 의사나...","긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활..."
2,응급질환,두통,예방,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...
3,응급질환,두통,예방,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 이러한 전략들은 ...,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...
4,응급질환,두통,예방,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...
...,...,...,...,...,...
5513,응급질환,두통,치료,"두통은 우리 일상생활에서 흔히 겪는 증상입니다. 만약 두통이 반복적으로 발생한다면,...",두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...
5514,응급질환,두통,치료,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...
5515,응급질환,두통,치료,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...
5516,응급질환,두통,치료,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...


### 키워드 및 문장기반 유사도 측정 및 질문-답변 매칭
(1) KeyBERT 모델을 사용해 키워드 유사도가 높은 질문들만 1차 추출
- 명사, 형용사 형태소로만 재 문장 구성 
- KeyBERT 모델을 활용해 문장의 키워드 추출(BERT 기반이므로 문맥정보 이해)
- TF-IDF 벡터로 변환 후 코사인 유사도 계산. 답변당 상위 n개의 질문만 추출
- 바로 모든 질문과 답변을 비교하기보다 1차 필터링을 통해 문장수를 줄여 대용량시 연산 속도를 높이기 위함.

(2) 필터링된 문장들만 임베딩 후 유사도 비교
- KoBERT: 한국어로 된 문장, 단어, 문맥을 이해하고 처리하는 범용 모델. ex) 감정 분석, 문서 분류, 질의 응답 등
- KR-SBERT: 문장 유사도에 조금 더 최적화

(3) 가장 연관성이 높은 질문 데이터프레임화

### (1)-1 형태소 분석 후 문장 재생성

In [4]:
from konlpy.tag import Okt

okt = Okt()

# 형태소 분석 및 품사 정보
test_keyword = []
for sentence in question_df['question']:
    test_keyword.append(okt.pos(sentence))

- 키워드 추출, 문장 재생성 함수

In [6]:
def text_extract(df, column):
    # 의도
    pos_keywords = []
    for q in df[f'{column}']:
        pos_keywords.append(okt.pos(q))

    # Noun, Adjective 키워드만 추출
    keyword_lists = []
    for list in pos_keywords:
        
        sentence_keywords = []
        for pair in list:
            keyword, pos = pair
            if pos in ['Noun', 'Adjective']:
                sentence_keywords.append((keyword, pos))

        keyword_lists.append(sentence_keywords)

    return keyword_lists

In [7]:
question_keyword_lists = text_extract(question_df, 'question')

In [8]:
# 다시 문장으로 만드는 함수 만들기
def remake_sentence(keyword_lists):
    remake_question = []
    for list in keyword_lists:
        sentence = ''
        for pair in list:
            sentence = sentence + pair[0] + ' '
        remake_question.append(sentence)

    return remake_question

In [9]:
question_remake_sentence = remake_sentence(question_keyword_lists)
question_remake_sentence[:5]

['두통 예방 위해 어떤 자세 활동 도움 요 ',
 '스트레스 사람 두통 예방 위해 어떤 생활 습관 요 ',
 '대상포진 예방접종 후 어지럼증 두통 예방접종 관련 있다면 어떻게 ',
 '갈색 포증 관련 일상생활 신경 점 무엇 ',
 '두통 예방 위 수면 자세 어떤 것 있을까요 ']

- 의도별 키워드 추출 및 문장 재생성 함수로 문장 재생성하기

In [10]:
question_df['intention'].unique()

array(['예방', '원인', '정의', '증상', '진단', '치료'], dtype=object)

In [11]:
# 키워드 추출 및 문장 재생성 함수
def remake_sentence_by_intention(df, intention, column):
    filtered_df = df[df['intention'] == f'{intention}']
    # 형태소 키워드 추출
    keyword_lists = text_extract(filtered_df, column)
    # 문장 재생성
    result = remake_sentence(keyword_lists)

    return result

In [12]:
len(remake_sentence_by_intention(question_df, '예방', 'question'))

308

In [14]:
intention_en = {'예방': 'prevention',
                '원인': 'cause',
                '정의': 'definition',
                '증상': 'symptom',
                '진단': 'diagnosis',
                '치료': 'treatment'}

for i in question_df['intention'].unique():
    print(f'{intention_en[i]}_sentence')

prevention_sentence
cause_sentence
definition_sentence
symptom_sentence
diagnosis_sentence
treatment_sentence


In [15]:
# 의도별 질문 문장 재생성
question_sentences_dic = {} # 딕셔너리로 변수명 저장

for i in question_df['intention'].unique():
    var_name = f'{intention_en[i]}_sentence' # 변수명
    question_sentences_dic[var_name] = remake_sentence_by_intention(question_df, i, 'question')

In [16]:
question_sentences_dic['prevention_sentence'][:3]

['두통 예방 위해 어떤 자세 활동 도움 요 ',
 '스트레스 사람 두통 예방 위해 어떤 생활 습관 요 ',
 '대상포진 예방접종 후 어지럼증 두통 예방접종 관련 있다면 어떻게 ']

In [17]:
# 의도별 답변 문장 재생성
answer_sentences_dic = {} # 딕셔너리로 변수명 저장

for i in answer_df['intention'].unique():
    var_name = f'{intention_en[i]}_sentence' # 변수명
    answer_sentences_dic[var_name] = remake_sentence_by_intention(answer_df, i, 'answer_intro_conclusion')

In [18]:
answer_sentences_dic['prevention_sentence'][:3]

['두통 예방 위 일상 자세 유지 일상 생활 자세한 관리 두통 예방 도움 수 있습니다 올바른 자세 수면 스트레칭 통해 일상 생활 올바른 자세 유지 것 두통 예방 도움 ',
 '긴장 두통 예방 위 몇 가지 조치 취할 수 있습니다 마지막 의사 영양사 상담 통해 적절한 예방 조치 취하 것 중요합니다 의사 영양사 조언 정기 건강 관리 스트레스 관리 긴장 두통 예방 데 도움 것 입니다 ',
 '두통 예방 위 생활 습관 다양한 방법 개선 수 있습니다 일상 습관 중 하나 충분한 수면 취하 것 입니다 수면 시간 충분히 확보 잠자리 전 긴장 운동 활동 것 좋습니다 또한 스트레스 관리 것 중요한 요소 입니다 스트레스 긴장 두통 유발 주요한 요인 므 스트레스 효과 관리 방법 것 중요합니다 두통 예방 위 일상 습관 개선 스트레스 관리 충분한 휴식 필요합니다 스트레스 높을 경우 의사 상담 적절한 치료 방법 것 좋습니다 ']

### (1)-2 keybert 모델로 키워드 추출
- 재생산한 문장에서의 키워드 중요도 반영

In [25]:
# !pip install --upgrade httpx

In [24]:
from keybert import KeyBERT

# 모델 불러오기
kw_model = KeyBERT()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Paris\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
list(question_sentences_dic.keys())

['prevention_sentence',
 'cause_sentence',
 'definition_sentence',
 'symptom_sentence',
 'diagnosis_sentence',
 'treatment_sentence']

In [27]:
question_sentences_dic.keys()

dict_keys(['prevention_sentence', 'cause_sentence', 'definition_sentence', 'symptom_sentence', 'diagnosis_sentence', 'treatment_sentence'])

In [28]:
# 질문 keybert 키워드 추출
question_keybert_dic = {}

# 변수명 딕셔너리에 키값으로 담음.
for key in list(question_sentences_dic.keys()):
    var = key.split('_')[0]
    var_name = f'{var}_question_keywords'

    question_keybert_dic[var_name] = []
    # question_keywords = []
    for q in question_sentences_dic[key]:
        kw = kw_model.extract_keywords(q, keyphrase_ngram_range=(1, 1), top_n=10)
        question_keybert_dic[var_name].append(kw)

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [29]:
# 제대로 들어갔나 확인
question_keybert_dic.keys(), question_keybert_dic['treatment_question_keywords'][:2]

(dict_keys(['prevention_question_keywords', 'cause_question_keywords', 'definition_question_keywords', 'symptom_question_keywords', 'diagnosis_question_keywords', 'treatment_question_keywords']),
 [[('진통제', 0.6325),
   ('충분한', 0.5314),
   ('두통', 0.4342),
   ('치료', 0.3565),
   ('효과', 0.3234),
   ('있을까요', 0.201),
   ('있을', 0.201)],
  [('성적', 0.5468),
   ('달성', 0.4924),
   ('장기', 0.444),
   ('두통', 0.4321),
   ('목표', 0.4252),
   ('도움', 0.2602),
   ('전략', 0.1802),
   ('예방', 0.1802),
   ('어떤', 0.1802)]])

In [30]:
# 답변 keybert 키워드 추출
answer_keybert_dic = {}

# 변수명 딕셔너리에 키값으로 담음.
for key in list(answer_sentences_dic.keys()):
    var = key.split('_')[0]
    var_name = f'{var}_answer_keywords'

    answer_keybert_dic[var_name] = []
    for q in answer_sentences_dic[key]:
        kw = kw_model.extract_keywords(q, keyphrase_ngram_range=(1, 1), top_n=10)
        answer_keybert_dic[var_name].append(kw)

In [31]:
# 제대로 들어갔나 확인
answer_keybert_dic.keys(), answer_keybert_dic['treatment_answer_keywords'][:2]

(dict_keys(['prevention_answer_keywords', 'cause_answer_keywords', 'definition_answer_keywords', 'symptom_answer_keywords', 'diagnosis_answer_keywords', 'treatment_answer_keywords']),
 [[('중요합니다', 0.5574),
   ('정신', 0.4489),
   ('적절한', 0.4295),
   ('사용', 0.4231),
   ('요법', 0.4121),
   ('치료', 0.4052),
   ('결정', 0.4021),
   ('생활', 0.3958),
   ('이완', 0.3898),
   ('전문가', 0.385)],
  [('정확한', 0.2811),
   ('성적', 0.2523),
   ('적절한', 0.2416),
   ('건강한', 0.2302),
   ('통증', 0.2273),
   ('중요합니다', 0.2206),
   ('라이프스타일', 0.2107),
   ('증상', 0.2028),
   ('의해', 0.1943),
   ('정기', 0.1932)]])

#### (1)-3 TF-IDF 벡터로 변환 후 코사인 유사도 계산
- TF-IDF 사용 이유: 유사한 [질문-> 답변]들이 있을경우 키워드간 중요도를 판별하기 위함.

In [ ]:
# # 변경전전
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # 키워드를 TF-IDF 벡터로 변환
# # 질문을 학습시킨 벡터화 모델에 답변을 벡터화해야 차원을 맞출 수 있음.
# # 질문만 학습시켜 생긴 어휘사전 기반으로 답변은 변환만 하면 어휘사전에 없는 단어들은 벡터화가 안됨.
# # 즉, 답변에만 등장하는 단어들의 유사도를 비교할 필요가 없는 것.
# # 답변이 질문보다 훨씬 길기 때문에 불필요한 단어들이 있을것이라 판단해 이렇게 벡터화함.
# vectorizer = TfidfVectorizer()

# def tfidf_vectorizer(q_keybert_dic, a_keybert_dic, q_column, a_column, vectorizer):
    
#     q_tfidf_dic = {}
#     a_tfidf_dic = {}
#     for q_key, a_key in zip(q_keybert_dic.keys(), a_keybert_dic.keys()):
#         var = q_key.split('_')[0]
#         q_var_name = f'{var}_{q_column}_vectors'
#         a_var_name = f'{var}_{a_column}_vectors'


#         q_tfidf_dic[q_var_name] = vectorizer.fit_transform([' '.join([kw[0] for kw in kblist]) for kblist in q_keybert_dic[q_key]])
#         a_tfidf_dic[a_var_name] = vectorizer.transform([' '.join([kw[0] for kw in kblist]) for kblist in a_keybert_dic[a_key]])

#     return q_tfidf_dic, a_tfidf_dic

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 키워드를 TF-IDF 벡터로 변환
# 답변을 학습시킨 벡터화 모델에 답변을 벡터화해야 차원을 맞출 수 있음.
# 답변만 학습시켜 생긴 어휘사전 기반으로 질문은 변환만 하면 어휘사전에 없는 단어들은 벡터화가 안됨.
# 즉, 질문에만 등장하는 단어들의 유사도를 비교할 필요가 없는 것.
# 답변이 질문보다 훨씬 길기 때문에 답변에 나온 키워드로 어휘사전을 만들고 질문을 변환함.
vectorizer = TfidfVectorizer()

def tfidf_vectorizer(q_keybert_dic, a_keybert_dic, q_column, a_column, vectorizer):
    
    q_tfidf_dic = {}
    a_tfidf_dic = {}
    for q_key, a_key in zip(q_keybert_dic.keys(), a_keybert_dic.keys()):
        var = q_key.split('_')[0]
        q_var_name = f'{var}_{q_column}_vectors'
        a_var_name = f'{var}_{a_column}_vectors'

        a_tfidf_dic[a_var_name] = vectorizer.fit_transform([' '.join([kw[0] for kw in kblist]) for kblist in a_keybert_dic[a_key]])
        q_tfidf_dic[q_var_name] = vectorizer.transform([' '.join([kw[0] for kw in kblist]) for kblist in q_keybert_dic[q_key]])

    return q_tfidf_dic, a_tfidf_dic

In [41]:
# tfi-df 벡터화 진행
question_vectors_dic, answer_vectors_dic = tfidf_vectorizer(question_keybert_dic, answer_keybert_dic, 'question', 'answer', vectorizer)

In [42]:
question_vectors_dic

{'prevention_question_vectors': <308x509 sparse matrix of type '<class 'numpy.float64'>'
 	with 1566 stored elements in Compressed Sparse Row format>,
 'cause_question_vectors': <567x562 sparse matrix of type '<class 'numpy.float64'>'
 	with 2303 stored elements in Compressed Sparse Row format>,
 'definition_question_vectors': <606x588 sparse matrix of type '<class 'numpy.float64'>'
 	with 2545 stored elements in Compressed Sparse Row format>,
 'symptom_question_vectors': <772x371 sparse matrix of type '<class 'numpy.float64'>'
 	with 2605 stored elements in Compressed Sparse Row format>,
 'diagnosis_question_vectors': <707x515 sparse matrix of type '<class 'numpy.float64'>'
 	with 3049 stored elements in Compressed Sparse Row format>,
 'treatment_question_vectors': <468x230 sparse matrix of type '<class 'numpy.float64'>'
 	with 1816 stored elements in Compressed Sparse Row format>}

In [44]:
print(answer_vectors_dic['symptom_answer_vectors'].shape) # 316개의 문장이 371개의 키워드로 표현됨.

(316, 371)


In [ ]:
# # 코사인 유사도 함수화
# # 증상 질문과 증상 답변, 진단 질문과 진단 답변들 간의 유사도
# # 코사인 유사도 결과 값을 보면 첫번째 데이터는 첫번째 답변과 모든 질문간의 유사도가 하나의 행렬 벡터에 있는 것이다.
# # 즉 각 벡터마다 답변 문장의 수만큼 유사도 점수가 있음.
# # 모든 답변 벡터에 대한 질문들의 유사도를 비교하고 싶으므로 답변 벡터를 먼저 배치.
# def from_dic_cos_similarity(q_vector_dic, a_vector_dic):

#     similarity_dic = {}
    
#     for q_key, a_key in zip(q_vector_dic.keys(), a_vector_dic.keys()):
#         var = q_key.split('_')[0]
#         var_name = f'{var}_keyword_similarities'

#         # 질문에 대한 
#         similarity_dic[var_name] = cosine_similarity(a_vector_dic[a_key], q_vector_dic[q_key])
    
#     return similarity_dic

In [ ]:
# 코사인 유사도 함수화
# 증상 질문과 증상 답변, 진단 질문과 진단 답변들 간의 유사도
# 코사인 유사도 결과 값을 보면 첫번째 데이터는 첫번째 질문과 모든 답변간의 유사도가 하나의 행렬 벡터에 있는 것이다.
# 즉 각 벡터마다 답변 문장의 수만큼 유사도 점수가 있음.
# 모든 질문 벡터에 대한 답변들의 유사도를 비교하고 싶으므로 질문 벡터를 먼저 배치.
def from_dic_cos_similarity(q_vector_dic, a_vector_dic):

    similarity_dic = {}
    
    for q_key, a_key in zip(q_vector_dic.keys(), a_vector_dic.keys()):
        var = q_key.split('_')[0]
        var_name = f'{var}_keyword_similarities'

        # 질문에 대한 
        similarity_dic[var_name] = cosine_similarity(q_vector_dic[q_key], a_vector_dic[a_key])
    
    return similarity_dic

In [46]:
keyword_similarities_dic = from_dic_cos_similarity(question_vectors_dic, answer_vectors_dic)

In [82]:
keyword_similarities_dic

{'prevention_keyword_similarities': array([[0.16677467, 0.        , 0.        , ..., 0.02665527, 0.03594705,
         0.        ],
        [0.05236798, 0.        , 0.04925338, ..., 0.02975389, 0.11306892,
         0.04714863],
        [0.04271667, 0.        , 0.        , ..., 0.03774474, 0.0509022 ,
         0.        ],
        ...,
        [0.04358659, 0.17856111, 0.        , ..., 0.03851341, 0.05193882,
         0.        ],
        [0.03921575, 0.        , 0.        , ..., 0.0346513 , 0.04673042,
         0.        ],
        [0.06040547, 0.        , 0.        , ..., 0.12067852, 0.16274592,
         0.        ]]),
 'cause_keyword_similarities': array([[0.2621224 , 0.31669515, 0.16640981, ..., 0.        , 0.27480986,
         0.24120924],
        [0.09149232, 0.11054062, 0.05808439, ..., 0.        , 0.0959208 ,
         0.0841927 ],
        [0.22667187, 0.36622495, 0.23876203, ..., 0.11400927, 0.31778897,
         0.20858709],
        ...,
        [0.12550994, 0.15164057, 0.07968066

In [69]:
# 모든 질문에 대한 답변을 매칭해야 하므로 질문의 개수만큼 유사도 리스트가 생성되야함.

print("데이터 프레임 '예방' 질문 개수: ", len(question_df[question_df['intention'] == '예방']))
print("'예방' 유사도 리스트 개수: ", len(keyword_similarities_dic['prevention_keyword_similarities']))

데이터 프레임 '예방' 질문 개수:  308
'예방' 유사도 리스트 개수:  308


In [80]:
# 각 증상 질문당 매칭된 답변의 유사도 개수
print('예방 첫번째 질문에 매칭된 답변 유사도 개수: ', len(keyword_similarities_dic['prevention_keyword_similarities'][0]))
print('예방 두번째 질문에 매칭된 답변 유사도 개수: ', len(keyword_similarities_dic['prevention_keyword_similarities'][1]))
print('원인 첫번째 질문에 매칭된 답변 유사도 개수: ', len(keyword_similarities_dic['cause_keyword_similarities'][0]))

예방 첫번째 질문에 매칭된 답변 유사도 개수:  1108
예방 두번째 질문에 매칭된 답변 유사도 개수:  1108
원인 첫번째 질문에 매칭된 답변 유사도 개수:  1328


In [71]:
len(keyword_similarities_dic['prevention_keyword_similarities'])

308

In [72]:
k = list(keyword_similarities_dic.keys())
for i in k:
    print(len(keyword_similarities_dic[i]))

308
567
606
772
707
468


In [73]:
k = list(keyword_similarities_dic.keys())
for i in k:
    if i == k[0]:
        print('ddd')
    else:
        print(i)

ddd
cause_keyword_similarities
definition_keyword_similarities
symptom_keyword_similarities
diagnosis_keyword_similarities
treatment_keyword_similarities


In [ ]:
# def top_n_similarity_idx(similarity_dic, top_n=5):

#     # key 리스트를 먼저 만들어야 마지막 for문에 사용할 수 있음.
#     key_lists = list(similarity_dic.keys())
#     answer_total_num = 0
#     question_total_num = 0
#     final_candidate_pairs =[]
    
#     for k in key_lists:
        
#         candidate_pairs = []
        
#         for i, row in enumerate(similarity_dic[k]):
#             idx_row=[]
#             for j, score in enumerate(row):
#                 # 유사도와 인덱스를 튜플로 만들어서 저장
#                 idx_row.append((j, score))
            
#             # reverse=True: 내림차순
#             # key: x[1]에 있는 값(=score)을 기준으로 정렬
#             top_sim = sorted(idx_row, reverse=True, key=lambda x: x[1])[:top_n]
#             for j, score in top_sim:
#                 if k == key_lists[0]:
#                     candidate_pairs.append((i, j))
#                 else:
#                     candidate_pairs.append((i + answer_total_num, j + question_total_num))
                
#         answer_total_num += len(similarity_dic[k])
#         question_total_num += len(similarity_dic[k][0])
#         final_candidate_pairs += candidate_pairs
                    

#     return final_candidate_pairs

In [83]:
def top_n_similarity_idx(similarity_dic, top_n=5):

    # key 리스트를 먼저 만들어야 마지막 for문에 사용할 수 있음.
    key_lists = list(similarity_dic.keys())
    answer_total_num = 0
    question_total_num = 0
    final_candidate_pairs =[]
    
    for k in key_lists:
        
        candidate_pairs = []
        
        for i, row in enumerate(similarity_dic[k]):
            idx_row=[]
            for j, score in enumerate(row):
                # 유사도와 인덱스를 튜플로 만들어서 저장
                idx_row.append((j, score))
            
            # reverse=True: 내림차순
            # key: x[1]에 있는 값(=score)을 기준으로 정렬
            top_sim = sorted(idx_row, reverse=True, key=lambda x: x[1])[:top_n]
            for j, score in top_sim:
                # 딕셔너리의 첫번째 key에 해당할때는 인덱스값 그대로 적용
                if k == key_lists[0]:
                    candidate_pairs.append((i, j))
                # 딕셔너리 두번째 key에 해당할때는 이전 key에 해당하는 데이터 개수만큼 더해줘야 전체 데이터에서의 인덱스를 설정해줄 수 있음
                else:
                    candidate_pairs.append((i + question_total_num, j + answer_total_num))
                
        question_total_num += len(similarity_dic[k])
        answer_total_num += len(similarity_dic[k][0])
        final_candidate_pairs += candidate_pairs
                    

    return final_candidate_pairs

In [84]:
final_candidate_pairs = top_n_similarity_idx(keyword_similarities_dic, top_n=20)

In [86]:
print('원천데이터 질문 개수: ', len(question_df))
print('최종 유사도 인덱스 쌍의 개수/n개: ', len(final_candidate_pairs) / 20)

원천데이터 질문 개수:  3428
최종 유사도 인덱스 쌍의 개수/n개:  3428.0


In [87]:
len(final_candidate_pairs)

68560

In [90]:
# 답변당 20개씩 매칭이 되어있음.
# (답변 위치(몇 번째 답변), 질문 위치)
final_candidate_pairs[6140:6170]

[(307, 387),
 (307, 247),
 (307, 36),
 (307, 753),
 (307, 318),
 (307, 805),
 (307, 305),
 (307, 826),
 (307, 861),
 (307, 10),
 (307, 657),
 (307, 710),
 (307, 213),
 (307, 857),
 (307, 840),
 (307, 338),
 (307, 434),
 (307, 63),
 (307, 976),
 (307, 468),
 (308, 2255),
 (308, 1954),
 (308, 2254),
 (308, 2286),
 (308, 2347),
 (308, 1868),
 (308, 1180),
 (308, 1625),
 (308, 2413),
 (308, 1344)]

### (2) 필터링된 문장들만 문장 임베딩
- KoBert 모델 사용

In [91]:
import torch
from kobert_transformers import get_kobert_model, get_tokenizer
model = get_kobert_model()
tokenizer = get_tokenizer()

# tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

In [93]:
# 임베딩 구하는 함수 만들기
def embedding(sentence_list, batch_size=32):
    
    # numpy 배열이 연산 속도가 빠르고 메모리 사용량이 적음.
    hidden_size = model.config.hidden_size  # BERT의 출력 차원 (예: 768). 단어 하나당 768개의 벡터로 표현
    all_embeddings = np.empty((len(sentence_list), hidden_size)) # 768개로 이루어진 배열 len(sentence)개 만큼 생성 

    # BERT 모델 최대 입력길이 제한이 512 토큰 이라고 함.
    for i in range(0, len(sentence_list), batch_size):
        
        # 임베딩하려는 문장을 batch 크기 만큼 묶어서 임베딩함.
        batch = sentence_list[i:i + batch_size]
        # 각 문장들을 토큰화함. 단어 개수대로 토큰화 되는게 아님.      
        tokens = tokenizer(batch, return_tensors='pt', max_length=512, truncation=True, padding=True)

        # 추론과정에서 기울기 구하지 않도록 설정
        # 학습이 아니라 추론만 하므로 메모리 사용량을 줄이고 속도를 높이기 위해 no_grad 사용
        with torch.no_grad():
            output = model(**tokens) # tokenizer 반환 형태 딕셔너리. **: 딕셔너리를 개별 키워드 값으로 전달.


        # [CLS]: 문장의 전체 의미를 요약하는 특수 토큰
        # output[0]: 각 문장의 cls(문장의 전체 의미) 토큰에 대한 임베딩 / (시퀀스 길이, 768)의 형태
        # shape: [batch_size, seq_length, hidden_size] / seq_length(시퀀스 길이): 각 문장마다 토큰화된 토큰 수
        # output[1]: 각 문장의 모든 토큰에 대한 임베딩
        # [:, 0, :]: 모든 batch 차원(한번에 처리하는 데이터 묶음), 시퀀스 길이에서 첫번째 토큰, 마지막 차원 모두
        batch_embeddings = output[0][:, 0, :].squeeze().numpy() # 유사도 비교를 위해 numpy 배열로 변환
        # numpy 배열에 바로 저장
        all_embeddings[i:i + batch_size] = batch_embeddings
    
    # 누적된 배치 임베딩을 하나의 큰 배열로 합침 
    return all_embeddings

In [94]:
question_embeddings_kobert = embedding(question_df['question'].tolist())

In [96]:
ic_answer_embeddings_kobert = embedding(answer_df['answer_intro_conclusion'].tolist())

In [97]:
answer_embeddings_kobert = embedding(answer_df['answer'].tolist())

In [65]:
# # 임베딩 구하는 함수 만들기
# def embedding(sentence):
#     # BERT 모델 최대 입력길이 제한이 512 토큰 이라고 함.
#     tokens = tokenizer.encode(sentence, return_tensors='pt', max_length=512, truncation=True, padding=True)

#     # 추론과정에서 기울기 구하지 않도록 설정
#     # 학습이 아니라 추론만 하므로 메모리 사용량을 줄이고 속도를 높이기 위해 no_grad 사용
#     with torch.no_grad():
#         output = model(tokens)


#     # output[0]: 각 입력 토큰에 대한 벡터
#     # shape: [batch_size, seq_length, hidden_size]
#     # output[1]: 문장 전체적인 의미
#     # [:, 0, :]: 모든 batch 차원(한번에 처리하는 데이터 묶음), 첫번째 토큰, 마지막 차원 모두
#     return output[0][:, 0, :].squeeze().numpy() # 유사도 비교를 위해 numpy 배열로 변환

In [66]:
# # 인트로+결론 답변, 전체 답변, 질문에 대한 임베딩 계산
# question_embeddings_kobert = [embedding(q) for q in question_df['question']] 

In [53]:
# ic_answer_embeddings_kobert = [embedding(a) for a in answer_df['answer_intro_conclusion']]

In [54]:
# answer_embeddings_kobert = [embedding(a) for a in answer_df['answer']]

In [99]:
len(question_embeddings_kobert), len(ic_answer_embeddings_kobert), len(answer_embeddings_kobert)

(3428, 5518, 5518)

In [103]:
# 후보 쌍에 대한 코싸인 유사도 계산 함수

def make_cosine_similarity(candidate_pairs, question_embedding, answer_embedding):
    
    similarities = []
    for q_idx, a_idx in candidate_pairs:
        # print(q_idx, a_idx)
        c_simi = cosine_similarity([question_embedding[q_idx]], [answer_embedding[a_idx]])
        similarities.append((q_idx, a_idx, c_simi))

    return similarities

In [ ]:
# 전체 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
kobert_final_similarities = make_cosine_similarity(final_candidate_pairs, question_embeddings_kobert, answer_embeddings_kobert)

In [105]:
len(kobert_final_similarities), kobert_final_similarities[1900]

(68560, (95, 252, array([[0.73021439]])))

In [106]:
# intro+conclusion 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
ic_kobert_final_similarities = make_cosine_similarity(final_candidate_pairs, question_embeddings_kobert, ic_answer_embeddings_kobert)

In [107]:
len(ic_kobert_final_similarities), ic_kobert_final_similarities[1900]

(68560, (95, 252, array([[0.77582595]])))

- 가중치를 부여한 유사도 점수 계산
- intro + conclusion 답변에 대한 유사도에 가중치 0.4, 전체 답변에 대한 유사도에 가중치 0.6 부여

In [108]:
ic_kobert_final_similarities[-1]

(3427, 5489, array([[0.63683665]]))

In [109]:
combine_similarities = []
for ic_s, s in zip(ic_kobert_final_similarities, kobert_final_similarities):
    a, b, ic_c = ic_s
    _, _, c = s

    combine_score = 0.4*ic_c + 0.6*c

    combine_similarities.append((a, b, combine_score))

kobert_final_similarities = combine_similarities

In [110]:
kobert_final_similarities[-1]

(3427, 5489, array([[0.59241638]]))

### (3) 가장 연관성이 높은 질문 데이터 프레임화

- 대량의 데이터를 계속 for문 속에 for문을 넣으면서 코드를 돌리니 시간이 너무 오래걸림.
- defaultdict를 사용해 한 번만 순회하면서 최대유사도를 업데이트하는 방향으로 코드 변경
- 160분 넘어도 안끝나던 코드가 -> 0.2초만에 끝남
- defaultdict
    - dictionary와 동일하게 key: value 형식.
    - 존재하지 않는 key에 대해 기본값을 설정할 수 있음.

In [82]:
# # 인덱스와 유사도 리스트화
# idx_group = set(i[0] for i in kobert_final_similarities) # set(): 유니크한 값을 가지는 집합

# for g in idx_group:
#     simi_list = [i[2] for i in kobert_final_similarities if i[0] == g]

#     # 유사도가 가장 높은 값에 해당하는 데이터 하나만 추출
#     result = [row for row in kobert_final_similarities if (row[2] == max(simi_list)) and (row[0]==g)] 

In [111]:
from collections import defaultdict
a = defaultdict(int)

In [113]:
a[0], a['키값'], a.values()

(0, 0, dict_values([0, 0]))

In [115]:
from collections import defaultdict

# 키에 대한 기본값 설정
# 어떤 키를 넣든 없으면 해당 기본값을 줌
max_group = defaultdict(lambda: (-1, None)) # max_group에 처음 접근할 때 키가 없으면 (-1(유사도), None(원래값))이 기본값

for row in kobert_final_similarities:
    q_idx, a_idx, simi = row
    if simi > max_group[q_idx][0]: # simi > 현재 할당되어있는 유사도 이면
        max_group[q_idx] = (simi, row) # 동일한 q_idx 키에대해 값 업데이트

kobert_final_result = [row for simi, row in max_group.values()]

In [119]:
len(kobert_final_result), kobert_final_result[3002]

(3428, (3002, 5419, array([[0.70191665]])))

In [168]:
question_df

,disease_category,disease_name,intention,question
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?
...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?


In [169]:
kobert_final_matching_list = []
for row in kobert_final_result:

    # 답변
    answer = answer_df.loc[row[1], 'answer']
    answer_intro_conclusion = answer_df.loc[row[1], 'answer_intro_conclusion']
    # 질문
    question = question_df.loc[row[0], 'question']
    # 유사도 점수
    similarity_score = row[2]
    # 질문 의도
    intent = question_df.loc[row[0], 'intention']
    # 질병관련 정보
    dc = answer_df.loc[row[1], 'disease_category']
    dn = answer_df.loc[row[1], 'disease_name']

    kobert_final_matching_list.append({'disease_category': dc, 'disease_name': dn,
                                       'intention': intent, 'question': question,
                                       # 'answer_intro_conclusion': answer_intro_conclusion,
                                       'answer': answer,
                                       'similarity_score': np.round(similarity_score, 4)})

In [170]:
kobert_final_matching_df = pd.DataFrame(kobert_final_matching_list)
kobert_final_matching_df[['question', 'answer', 'similarity_score']]

,question,answer,similarity_score
0,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,[[0.6894]]
1,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",[[0.6251]]
2,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,[[0.6988]]
3,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,[[0.7173]]
4,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",[[0.6314]]
...,...,...,...
3423,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...,[[0.7376]]
3424,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,[[0.7367]]
3425,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,[[0.748]]
3426,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통 치료에는 다양한 방법이 있습니다. 약물 치료는 일반적인 원인 질환에 따라 사용...,[[0.6547]]


In [171]:
kobert_final_matching_df['similarity_score'] = kobert_final_matching_df['similarity_score'].apply(lambda x: x[0][0])

In [172]:
kobert_final_matching_df.head(2)

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251


In [147]:
kobert_final_matching_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3428 entries, 0 to 3427
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   disease_category  3428 non-null   object 
 1   disease_name      3428 non-null   object 
 2   intention         3428 non-null   object 
 3   question          3428 non-null   object 
 4   answer            3428 non-null   object 
 5   similarity_score  3428 non-null   float64
dtypes: float64(1), object(5)
memory usage: 160.8+ KB


In [188]:
kobert_final_matching_df['question'][3]

'갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?'

In [189]:
kobert_final_matching_df['answer'][3].split('. ')

['만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다',
 '스트레스와 긴장으로 인해 발생하는 두통은 심각할 수 있습니다',
 '만약 계속해서 두통이 지속되고 있다면, 의사에게 상담하여 증상의 원인을 확인하는 것이 중요합니다',
 '일반적으로 의사는 뇌혈류 검사를 통해 두통의 원인을 확인할 수 있습니다',
 '또한, 두통의 증상과 기간을 평가하고, 환자의 생활 습관과 개인적인 요인을 조사합니다',
 '두통을 효과적으로 관리하기 위해 진통제와 예방제를 사용할 수 있습니다',
 '그러나 예방을 위해 규칙적인 식습관과 충분한 수면, 스트레스 관리 등의 생활습관 개선이 필요합니다',
 '스트레스는 뇌혈류의 변화와 관련이 있으므로, 두통이 지속될 경우 의사와 상담하여 예방 조치를 취할 수 있습니다.']

In [175]:
kobert_final_matching_df.to_csv('kobert_matching.csv')

In [4]:
kobert_final_matching_df = pd.read_csv('kobert_matching.csv', index_col=0)
kobert_final_matching_df

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.6988
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.7173
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6314
...,...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...,0.7376
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7367
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7480
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통 치료에는 다양한 방법이 있습니다. 약물 치료는 일반적인 원인 질환에 따라 사용...,0.6547


### (2)-1 KoSentence-BERT 모델로 문장 유사도 다시 구해보기

In [152]:
from sentence_transformers import SentenceTransformer, util

# KoSentence-BERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [153]:
# 임베딩 구하는 함수 만들기
def embedding_KR_SBERT(sentence, batch_size=32):
    result = model.encode(sentence, batch_size=batch_size)
    
    return result

In [154]:
# 질문과 답변에 대한 임베딩 계산
question_embeddings_KR_SBERT = embedding_KR_SBERT(question_df['question'].tolist())

In [155]:
answer_embeddings_KR_SBERT = [embedding_KR_SBERT(a) for a in answer_df['answer']]

In [ ]:
ic_answer_embeddings_KR_SBERT = [embedding_KR_SBERT(a) for a in answer_df['answer_intro_conclusion']]

In [158]:
# 후보 쌍에 대한 코싸인 유사도 계산 함수

def make_cosine_similarity(candidate_pairs, question_embedding, answer_embedding):
    
    similarities = []
    for q_idx, a_idx in candidate_pairs:
        # print(q_idx, a_idx)
        c_simi = cosine_similarity([question_embedding[q_idx]], [answer_embedding[a_idx]])
        similarities.append((q_idx, a_idx, c_simi))

    return similarities

In [162]:
# 후보 쌍에 대해 코싸인 유사도 계산
KR_SBERT_final_similarities = make_cosine_similarity(final_candidate_pairs, question_embeddings_KR_SBERT, answer_embeddings_KR_SBERT)

In [163]:
len(KR_SBERT_final_similarities), KR_SBERT_final_similarities[1900]

(68560, (95, 252, array([[0.50480473]], dtype=float32)))

In [164]:
# intro+conclusion 답변과 질문
# 후보 쌍에 대해 코싸인 유사도 계산
ic_KR_SBERT_final_similarities = make_cosine_similarity(final_candidate_pairs, question_embeddings_KR_SBERT, ic_answer_embeddings_KR_SBERT)

In [165]:
KR_SBERT_combine_similarities = []
for ic_s, s in zip(ic_KR_SBERT_final_similarities, KR_SBERT_final_similarities):
    a, b, ic_c = ic_s
    _, _, c = s

    combine_score = 0.4*ic_c + 0.6*c

    KR_SBERT_combine_similarities.append((a, b, combine_score))

KR_SBERT_final_similarities = KR_SBERT_combine_similarities
KR_SBERT_final_similarities[1900]

(95, 252, array([[0.500963]], dtype=float32))

In [166]:
from collections import defaultdict

# 키에 대한 기본값 설정
# 어떤 키를 넣든 없으면 해당 기본값을 줌
max_group = defaultdict(lambda: (-1, None)) # max_group에 처음 접근할 때 키가 없으면 (-1(유사도), None(원래값))이 기본값

for row in KR_SBERT_final_similarities:
    q_idx, a_idx, simi = row
    if simi > max_group[q_idx][0]: # simi > 현재 할당되어있는 유사도 이면
        max_group[q_idx] = (simi, row) # 동일한 a_idx 키에대해 값 업데이트

KR_SBERT_final_result = [row for simi, row in max_group.values()]

In [167]:
len(KR_SBERT_final_result)

3428

In [177]:
KR_SBERT_final_matching_list = []
for row in KR_SBERT_final_result:

    # 답변
    answer = answer_df.loc[row[1], 'answer']
    answer_intro_conclusion = answer_df.loc[row[1], 'answer_intro_conclusion']
    # 질문
    question = question_df.loc[row[0], 'question']
    # 유사도 점수
    similarity_score = row[2]
    # 질문 의도
    intent = question_df.loc[row[0], 'intention']
    # 질병관련 정보
    dc = answer_df.loc[row[1], 'disease_category']
    dn = answer_df.loc[row[1], 'disease_name']

    KR_SBERT_final_matching_list.append({'disease_category': dc, 'disease_name': dn,
                                         'intention': intent, 'question': question,
                                         # 'answer_intro_conclusion':answer_intro_conclusion,
                                         'answer': answer,
                                         'similarity_score': np.round(similarity_score, 4)})

In [178]:
KR_SBERT_final_matching_df = pd.DataFrame(KR_SBERT_final_matching_list)
KR_SBERT_final_matching_df[['question', 'answer', 'similarity_score']]

,question,answer,similarity_score
0,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,[[0.7519]]
1,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,[[0.7753]]
2,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,[[0.5899]]
3,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",[[0.4367]]
4,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,[[0.775]]
...,...,...,...
3423,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,"두통은 원인에 따라 다양한 치료 방법을 사용할 수 있습니다. 한편, 두통의 원인을 ...",[[0.754]]
3424,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,[[0.6936]]
3425,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,[[0.7687]]
3426,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...,[[0.7581]]


In [179]:
KR_SBERT_final_matching_df['similarity_score'] = KR_SBERT_final_matching_df['similarity_score'].apply(lambda x: x[0][0])

In [180]:
KR_SBERT_final_matching_df.head(2)

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753


In [181]:
KR_SBERT_final_matching_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3428 entries, 0 to 3427
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   disease_category  3428 non-null   object 
 1   disease_name      3428 non-null   object 
 2   intention         3428 non-null   object 
 3   question          3428 non-null   object 
 4   answer            3428 non-null   object 
 5   similarity_score  3428 non-null   float32
dtypes: float32(1), object(5)
memory usage: 147.4+ KB


In [190]:
KR_SBERT_final_matching_df['question'][3]

'갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?'

In [192]:
KR_SBERT_final_matching_df['answer'][3].split('. ')

['긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은 그와 관련된 통증을 관리하는 것입니다',
 '적절한 자세를 유지하고 스트레스를 관리하는 것이 중요합니다',
 '스트레스는 통증의 원인이 될 수 있으므로, 이를 예방하기 위해 정기적인 운동이나 명상 등을 실천하는 것이 좋습니다',
 '일상적인 활동을 향상시키고 피로를 피하기 위해 충분한 수면을 취하는 것도 도움이 됩니다',
 '또한, 정기적인 건강 검진을 받고 의사의 지시에 따라 약물이나 보조적인 치료를 사용하는 것도 중요합니다',
 '스트레스 관리와 충분한 수면, 건강한 식습관과 정기적인 운동, 충분한 수분 섭취 등은 스트레스와 통증을 효과적으로 관리하는 데 도움이 됩니다.']

In [193]:
KR_SBERT_final_matching_df.to_csv('KR_SBERT_matching.csv')

In [3]:
KR_SBERT_final_matching_df = pd.read_csv('KR_SBERT_matching.csv', index_col=0)
KR_SBERT_final_matching_df

,disease_category,disease_name,intention,question,answer,similarity_score
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,0.5899
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",0.4367
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,0.7750
...,...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,"두통은 원인에 따라 다양한 치료 방법을 사용할 수 있습니다. 한편, 두통의 원인을 ...",0.7540
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.6936
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7687
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...,0.7581


----

### KoBert 모델과 KR-SBERT 모델 결과 비교

In [11]:
# df = pd.merge(kobert_final_matching_df, KR_SBERT_final_matching_df, on=['question', 'intention'], how= 'left')
# df.columns

In [12]:
# model_result_df = df[['disease_category_x', 'disease_name_x', 'intention', 'question', 'answer_x', 'similarity_score_x', 'answer_y', 'similarity_score_y']]

In [13]:
# model_result_df.rename(columns={'disease_category_x': 'disease_category',
#                                 'disease_name_x': 'disease_name',
#                                 'answer_x': 'kobert_answer',
#                                 'similarity_score_x': 'kobert_similarity_score',
#                                 'answer_y': 'kr_sbert_answer',
#                                 'similarity_score_y': 'kr_sbert_similarity_score'
#                                 }, inplace=True)

In [14]:
# model_result_df.head(5)

In [15]:
# model_result_df['question'][4]

In [16]:
# model_result_df['kobert_answer'][4].split('. ')

In [17]:
# model_result_df['kr_sbert_answer'][4].split('. ')

---------------

### 평균과 표준편차를 맞춘 후 유사도 비교

In [5]:
kobert_mean, kobert_std = np.mean(kobert_final_matching_df['similarity_score']), np.std(kobert_final_matching_df['similarity_score'])
krsbert_mean, krsbert_std = np.mean(KR_SBERT_final_matching_df['similarity_score']), np.std(KR_SBERT_final_matching_df['similarity_score'])

print(f"[kobert]\n평균: {kobert_mean}\n표준편차: {kobert_std}")
print(f"[KRSBERT]\n평균: {krsbert_mean}\n표준편차: {krsbert_std}")

[kobert]
평균: 0.6773873687281213
표준편차: 0.06448183213386632
[KRSBERT]
평균: 0.6611920945157526
표준편차: 0.11255157624388429


- 표준편차가 약 2배정도 차이남. (두 모델의 유사도 계산 방식이 다르기 때문)
- 정규화를 통해 유사도 점수를 비교 가능하게함.

In [6]:
# 평균 0, 표준편차가 1인 형태로 정규화
kobert_final_matching_df['similarity_norm'] = (kobert_final_matching_df['similarity_score'] - kobert_mean) / kobert_std
KR_SBERT_final_matching_df['similarity_norm'] = (KR_SBERT_final_matching_df['similarity_score'] - krsbert_mean) / krsbert_std

In [7]:
kobert_final_matching_df.head()

,disease_category,disease_name,intention,question,answer,similarity_score,similarity_norm
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894,0.186295
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251,-0.810885
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.6988,0.332072
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.7173,0.618975
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6314,-0.713183


In [10]:
KR_SBERT_final_matching_df.head()

,disease_category,disease_name,intention,question,answer,similarity_score,similarity_norm
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519,0.805923
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753,1.013828
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,0.5899,-0.633417
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",0.4367,-1.994571
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,0.7750,1.011162


In [36]:
ko_df = kobert_final_matching_df.add_prefix('ko_') # 컬럼명 수정
kr_df = KR_SBERT_final_matching_df.add_prefix('kr_')
concat_df = pd.concat([ko_df, kr_df], axis=1)
concat_df

,ko_disease_category,ko_disease_name,ko_intention,ko_question,ko_answer,ko_similarity_score,ko_similarity_norm,kr_disease_category,kr_disease_name,kr_intention,kr_question,kr_answer,kr_similarity_score,kr_similarity_norm
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894,0.186295,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519,0.805923
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251,-0.810885,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753,1.013828
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.6988,0.332072,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,0.5899,-0.633417
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.7173,0.618975,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",0.4367,-1.994571
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6314,-0.713183,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,0.7750,1.011162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...,0.7376,0.933792,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,"두통은 원인에 따라 다양한 치료 방법을 사용할 수 있습니다. 한편, 두통의 원인을 ...",0.7540,0.824581
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7367,0.919835,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.6936,0.287938
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7480,1.095078,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7687,0.955188
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통 치료에는 다양한 방법이 있습니다. 약물 치료는 일반적인 원인 질환에 따라 사용...,0.6547,-0.351841,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...,0.7581,0.861009


In [37]:
concat_df.columns

Index(['ko_disease_category', 'ko_disease_name', 'ko_intention', 'ko_question',
       'ko_answer', 'ko_similarity_score', 'ko_similarity_norm',
       'kr_disease_category', 'kr_disease_name', 'kr_intention', 'kr_question',
       'kr_answer', 'kr_similarity_score', 'kr_similarity_norm'],
      dtype='object')

In [42]:
model_result_df = concat_df[['ko_disease_category', 'ko_disease_name', 'ko_intention', 'ko_question', 
                            'ko_answer', 'ko_similarity_score', 'ko_similarity_norm', 
                            'kr_question', 'kr_answer', 'kr_similarity_score', 'kr_similarity_norm'
                            ]]

In [43]:
model_result_df.rename(columns={'ko_disease_category': 'disease_category',
                                'ko_disease_name': 'disease_name',
                                'ko_intention': 'intention',
                                'ko_question': 'question',
                                'ko_similarity_score': 'kobert_similarity_score',
                                'kr_similarity_score': 'kr_sbert_similarity_score'
                                }, inplace=True)

In [44]:
model_result_df

,disease_category,disease_name,intention,question,ko_answer,kobert_similarity_score,ko_similarity_norm,kr_question,kr_answer,kr_sbert_similarity_score,kr_similarity_norm
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894,0.186295,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519,0.805923
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251,-0.810885,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753,1.013828
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.6988,0.332072,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,0.5899,-0.633417
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.7173,0.618975,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",0.4367,-1.994571
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6314,-0.713183,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,0.7750,1.011162
...,...,...,...,...,...,...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...,0.7376,0.933792,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,"두통은 원인에 따라 다양한 치료 방법을 사용할 수 있습니다. 한편, 두통의 원인을 ...",0.7540,0.824581
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7367,0.919835,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.6936,0.287938
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7480,1.095078,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...,0.7687,0.955188
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통 치료에는 다양한 방법이 있습니다. 약물 치료는 일반적인 원인 질환에 따라 사용...,0.6547,-0.351841,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...,0.7581,0.861009


In [45]:
model_result_df['final_answer'] = np.where(model_result_df['ko_similarity_norm'] > model_result_df['kr_similarity_norm'], model_result_df['ko_answer'], model_result_df['kr_answer'])
model_result_df.head(5)

,disease_category,disease_name,intention,question,ko_answer,kobert_similarity_score,ko_similarity_norm,kr_question,kr_answer,kr_sbert_similarity_score,kr_similarity_norm,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.6894,0.186295,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...,0.7519,0.805923,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6251,-0.810885,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...,0.7753,1.013828,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.6988,0.332072,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,긴장성 두통은 일상 생활에서 두통을 유발할 수 있는 스트레스와 관련된 원인으로 발생...,0.5899,-0.633417,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...,0.7173,0.618975,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,"긴장성 두통은 일상생활에서 신경이 쓰이는 통증을 유발하는 상황으로, 예방하는 방법은...",0.4367,-1.994571,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,"두통의 예방법에는 몇 가지 방법이 있습니다. 첫째로, 일상생활에서 올바른 자세를 유...",0.6314,-0.713183,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...,0.7750,1.011162,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...


In [46]:
model_result_df = model_result_df[['disease_category', 'disease_name', 'intention', 'question', 'final_answer']]
model_result_df

,disease_category,disease_name,intention,question,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...
...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...


In [47]:
model_result_df.to_csv('headache_qa_matching.csv')

In [48]:
model_result_df = pd.read_csv('headache_qa_matching.csv', index_col=0)
model_result_df

,disease_category,disease_name,intention,question,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...
...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...
